In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Read the text file
with open('F:/Projects/ML(AK)/Next Word Prediction Model using Python/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [8]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Force model building by specifying the input shape
model.build(input_shape=(None, max_sequence_len-1))

# Print the summary
print(model.summary())

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 17, 100)        │       820,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8200)           │     1,238,200 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,208,800 (8.43 MB)

 Trainable params: 2,208,800 (8.43 MB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 124s 40ms/step - accuracy: 0.0641 - loss: 6.5305
Epoch 2/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 136s 38ms/step - accuracy: 0.1205 - loss: 5.5420
Epoch 3/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 116s 38ms/step - accuracy: 0.1467 - loss: 5.1187
Epoch 4/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 120s 40ms/step - accuracy: 0.1656 - loss: 4.7659
Epoch 5/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 133s 37ms/step - accuracy: 0.1850 - loss: 4.4459
Epoch 6/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 116s 39ms/step - accuracy: 0.2068 - loss: 4.1392
Epoch 7/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 122s 41ms/step - accuracy: 0.2341 - loss: 3.8526
Epoch 8/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 124s 41ms/step - accuracy: 0.2679 - loss: 3.5933
Epoch 9/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 126s 42ms/step - accuracy: 0.3015 - loss: 3.3448
Epoch 10/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 142s 42ms/step - accuracy: 0.3379 - loss: 3.1135
Epoch 11/20
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 118s 39ms/step - accuracy: 0.3790 - loss: 2.89

In [10]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
I will leave if they may have told
